In [1]:
import tensorflow as tf
from Memory import Memory
from Experience import Experience
from Map import Map
import novelty
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow import keras
import numpy as np
from PIL import Image
import os
from typing import List
from ArtificialCuriosityTypes import ArtificialCuriosityTypes as act
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'



from matplotlib.pyplot import imshow

In [29]:
class New_Brain:
    def __init__(self, nov_thresh: float, novelty_function: novelty):
        self.nov_thresh = nov_thresh
        self.novelty_function = novelty_function
        self._init_CNN()
        self._init_AE()

    def _init_CNN(self):
        self._CNN_Base = keras.applications.VGG16(include_top=True)
        self._CNN_Base.trainable = False
        self._CNN = keras.Model(inputs=self._CNN_Base.layers[0].input, outputs=self._CNN_Base.layers[-2].input)

    def _init_AE(self):
        # vgg output is 4096
        input_vec = layers.Input(shape=(4096,))
        encoder1 = layers.Dense(2048, 'sigmoid')(input_vec)
        encoder2 = layers.Dense(1024, 'sigmoid')(encoder1)
        encoder3 = layers.Dense(512, 'sigmoid')(encoder2)
        # decoder
        decoder1 = layers.Dense(1024, 'sigmoid')(encoder3)
        decoder2 = layers.Dense(2048, 'sigmoid')(decoder1)
        decoder3 = layers.Dense(4096, 'sigmoid')(decoder2)

        self._AE = Model(inputs=input_vec, outputs=decoder3)

    def _prep_single_grain(self, grain):
        grain = keras.preprocessing.image.img_to_array(grain)
        grain = tf.image.per_image_standardization(grain)
        grain = tf.image.resize(grain, (224, 224))
        grain = tf.reshape(grain, (224, 224, 3))

        return grain

    def _prep_directional_grains(self, grains):
        # right now set to 4, but will probably want more in the future
        return tf.data.Dataset.from_tensor_slices(list(map(self._prep_single_grain, grains))).batch(4)

    def _CNN_process_grains(self, vect_images):
        return self._CNN(list(iter(vect_images))[0])

    def _run_through_main_model(self, grains):
        vect_images = self._prep_directional_grains(grains)
        feature_vects = self._CNN(list(iter(vect_images))[0])
        reconstructed = self._AE(feature_vects)
        return list(zip(feature_vects, reconstructed))

    def eval_novelty(self, grains: List[act.Grain]):
        pass

    def learn_grains(self, grains: List[act.Grain], memory: Memory = None):
        pass

In [4]:
nb = New_Brain(0.5, None)

In [5]:
im = Image.open("x.jpg")

In [6]:
nb._run_through_main_model([im, im, im, im])

[(<tf.Tensor: shape=(4096,), dtype=float32, numpy=
  array([0.       , 1.1585238, 0.       , ..., 0.       , 1.401871 ,
         0.       ], dtype=float32)>,
  <tf.Tensor: shape=(4096,), dtype=float32, numpy=
  array([0.50530785, 0.51404357, 0.51217115, ..., 0.4884638 , 0.49615696,
         0.46672434], dtype=float32)>),
 (<tf.Tensor: shape=(4096,), dtype=float32, numpy=
  array([0.       , 1.1585238, 0.       , ..., 0.       , 1.401871 ,
         0.       ], dtype=float32)>,
  <tf.Tensor: shape=(4096,), dtype=float32, numpy=
  array([0.50530785, 0.51404357, 0.51217115, ..., 0.4884638 , 0.49615696,
         0.46672434], dtype=float32)>),
 (<tf.Tensor: shape=(4096,), dtype=float32, numpy=
  array([0.       , 1.1585238, 0.       , ..., 0.       , 1.401871 ,
         0.       ], dtype=float32)>,
  <tf.Tensor: shape=(4096,), dtype=float32, numpy=
  array([0.50530785, 0.51404357, 0.51217115, ..., 0.4884638 , 0.49615696,
         0.46672434], dtype=float32)>),
 (<tf.Tensor: shape=(4096,), dt

In [3]:
from New_Brain import New_Brain